# Scrape Geschäfte

In [1]:
import urllib
import calendar
import requests
from pathlib import Path
import glob, os
import xml.etree.ElementTree as ET
import pandas as pd
import json
import utils
from dateutil import parser

## 1. Load all Geschäfte

In [2]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh5/cdws/Index/GESCHAEFT/searchdetails?q=%s&l=de-CH'

year_from = 2001
year_to = 2022
print("Years: %s - %s" % (year_from, year_to))
for year in range(year_from, year_to):
    for i in range(0, 3):
        #print("%s %s" % (year, i))

        # First day
        d_start = '%s-%02d-01 00:00:00' % (year, i * 4 + 1)

        # Last day
        l = calendar.monthrange(year, (i + 1) * 4)[-1]
        d_end = '%s-%02d-%s 23:59:00' % (year, (i + 1) * 4, l)

        q = 'beginn_start >= "%s" and beginn_end <= "%s" sortBy beginn_start/sort.descending krnr/sort.descending' % (d_start, d_end)

        r = requests.get(url % urllib.parse.quote(q))
        with open(Path('../export/GESCHAEFT/%s_%s.xml' % (year, i)), 'wb') as f:
            f.write(r.content)


Years: 2001 - 2022


## 2. Create DataFrame from XML

In [2]:
# Parse all XML and create Dataframe
ns = {
    'g': 'http://www.cmiag.ch/cdws/Geschaeft'
    }

records = []

# Open all xml-files
for f in glob.glob(str(Path('../export/GESCHAEFT/*.xml'))):
    tree = ET.parse(f)
    root = tree.getroot()

    try:

        for g in root.findall('.//g:Geschaeft', ns):

            if g.find('g:PendentBeiNeu/g:Organisationseinheit/g:Name', ns) == None:
                continue

            # Find Erstunterzeichner:in
            erstunterzeichner = {'name': '', 'vorname': '', 'istkantonsrat': False, 'partei': ''}
            for beteiligung in g.findall('g:Beteiligungen/g:Beteiligung', ns):
                if (beteiligung.find('g:Rolle', ns).text != None) and ('erstunterzeichner' in beteiligung.find('g:Rolle', ns).text.lower()):
                    erstunterzeichner['vorname'] = beteiligung.find('g:Vorname', ns).text
                    erstunterzeichner['name'] = beteiligung.find('g:Name', ns).text
                    erstunterzeichner['istkantonsrat'] = beteiligung.find('g:IstKantonsrat', ns).text
                    erstunterzeichner['partei'] = beteiligung.find('g:ParteiKurzname', ns).text

                    if type(erstunterzeichner['istkantonsrat']) == str:
                      erstunterzeichner['istkantonsrat'] = True if erstunterzeichner['istkantonsrat'].lower() == 'true' else False
                    break

            # Find last Ablaufschritt
            ablaufschritte = map(lambda x: {
                'Start': parser.parse(x.find('g:Sitzungsdatum/g:Start', ns).text),
                'AblaufschrittTyp': x.find('g:AblaufschrittTyp', ns).text,
                'StatusText': x.find('g:StatusText', ns).text,
                }, g.findall('g:Ablaufschritte/g:Ablaufschritt', ns))

            ablaufschritte = sorted(ablaufschritte, key = lambda x: x['Start'], reverse = True)

            # Add Record
            records.append({
                'krnr': g.find('g:KRNr', ns).text,
                'vorlagenr': g.find('g:VorlageNr', ns).text,
                'titel': g.find('g:Titel', ns).text,
                'geschaeftsart': g.find('g:Geschaeftsart', ns).text,
                'behandelndekommission': g.find('g:BehandelndeKommission', ns).text,
                'behandelndekommissionkurzname': g.find('g:BehandelndeKommissionKurzname', ns).text,
                'direktion': g.find('g:Direktion', ns).text,
                'direktionKurzname': g.find('g:DirektionKurzname', ns).text,
                'start': parser.parse(g.find('g:Beginn/g:Start', ns).text),
                'end': parser.parse(g.find('g:Beginn/g:End', ns).text),
                'zusammenfassung': g.find('g:Zusammenfassung', ns).text,
                'status': g.find('g:PendentBeiNeu/g:Organisationseinheit/g:Name', ns).text,
                'erstunterzeichnervorname': erstunterzeichner['vorname'],
                'erstunterzeichnername': erstunterzeichner['name'],
                'erstunterzeichneristkantonsrat': erstunterzeichner['istkantonsrat'],
                'erstunterzeichnerpartei': erstunterzeichner['partei'],
                'letzterschrittstart': ablaufschritte[0]['Start'],
                'letzterschritttyp': ablaufschritte[0]['AblaufschrittTyp'],
                'letzterschritttext': ablaufschritte[0]['StatusText'],
            })

    except:
        print(f)
        print(g.find('g:KRNr', ns).text)
        raise

df_raw = pd.DataFrame(records)
df_raw = df_raw.sort_values('start')

## Join with Member Data

In [3]:
with open(Path('../export/mitglieder.json'), encoding='utf-8') as f:
    mitglieder = json.load(f)

df_mitglieder = pd.DataFrame(list(map(lambda m: {'name': m['name'], 'vorname': m['vorname'], 'geschlecht': m['geschlecht'], 'jahrgang': m['jahrgang']}, mitglieder)))

In [4]:
df = df_raw.copy()
df['_name'] = df.apply(lambda row: "%s %s" % (row['erstunterzeichnervorname'], row['erstunterzeichnername']), axis=1)

# Replace some names with Aliases
for i, row in df.iterrows():
    for alias in utils.aliases:
        if alias[0].lower() == row['_name'].lower():
            df.loc[i, 'erstunterzeichnername'] = alias[2]
            df.loc[i, 'erstunterzeichnervorname'] = alias[1]
            break

In [5]:
df = df.merge(df_mitglieder, left_on = ['erstunterzeichnervorname', 'erstunterzeichnername'], right_on = ['vorname', 'name'], how='left')
df.drop(columns=['name', 'vorname', '_name'], inplace=True)

df.to_csv(Path('../export/geschaefte.csv'), index=False)